In [21]:
# 改名客户统计
from imp4nb import *

cnx = lite.connect('data\\quandan.db')
# desclitedb(cnx)

cur = cnx.cursor()
result = cur.execute('select count(*) from xiaoshoumingxi')
print(result.fetchone()[0])

df = pd.read_sql_query(
    "select 日期,职员名称 as 业务主管,单据编号 as 销售单号,商品备注 as 订单单号,单据类型,摘要,xiaoshoumingxi.单位全名 as 客户,customer.往来单位编号 as 编码,"
    "xiaoshoumingxi.商品全名 as 商品全名,数量,xiaoshoumingxi.单价,金额,substr(customer.往来单位编号,1,2) as 区域 ,"
    "substr(customer.往来单位编号,12,1) as 类型,product.品牌名称  as 品牌 from xiaoshoumingxi,"
    "customer,product where (customer.往来单位 = xiaoshoumingxi.单位全名) "
    "and (product.商品全名 = xiaoshoumingxi.商品全名) order by 日期 desc", cnx)
descdb(df)

cnx.close()

1397275


In [17]:
from imp4nb import *

cnx = lite.connect('data\\quandan.db')

df = pd.read_sql_query('select * from xiaoshoumingxi',cnx)
# descdb(df)

dfcm = df.duplicated()
print(list(dfcm[dfcm == True].index))
df =df.drop_duplicates()
# descdb(df)

cnx.close()

[]
